# Phase 1: Problem Definition

**Topic**: Optimal Item-Level Discount Targeting for Churn Prevention

**Course**: Stanford CME 241 – Winter 2026

**Date**: February 9, 2026

## Problem Description and Relevance

In the online retail economy, customer retention is often prioritized over short-term revenue. However, aggressive retention strategies, such as frequent deep discounting, can lead to **Discount Addiction** (formally known as the Reference Price Effect). When customers receive frequent price cuts, their internal reference price lowers, and they may eventually refuse to purchase at full price, eroding long-term customer Lifetime Value (LTV).

### Design Constraint: Fixed Discount Depth

To reduce the complexity of the action space and limit regulatory exposure from fully personalized pricing, we fix the discount depth to a constant $\delta$ (e.g., 30%) for all promotions. The agent's only decision is *which product to promote* (if any) — not how large the discount should be. This shifts the optimization problem from *how much to charge* to *what item to feature*.

### Practical Challenge

We aim to build a recommendation engine that balances three conflicting objectives:

- **Churn Risk**: Offering a discount on a high-affinity item reduces the probability of the customer leaving (terminal state). Preventing churn is the key driver, as active customers continue to engage and generate revenue.
- **Discount Addiction**: Frequent discounts lower the customer's internal reference price. Discount too often and the customer refuses to buy at full price, permanently eroding margins.
- **Cannibalization**: Discounting an item the customer would have bought at full price wastes margin without generating an incremental sale.

### Data

We use the *Dunnhumby — The Complete Journey* dataset [1] to calibrate the model parameters:

> This dataset contains household level transactions over two years from a group of 2,500 households who are frequent shoppers at a retailer. It contains all of each household's purchases, not just those from a limited number of categories.

We cluster products into $N$ categories to define a tractable action space. We assume products in the same category are perfect substitutes and the probability of purchase for products in different categories are perfectly independent.

## MDP Formulation

We model the problem as a discounted, infinite-horizon MDP $(\mathcal{S}, \mathcal{A}, P, R, \gamma)$. At each discrete time step $t = 0, 1, 2, \ldots$ the agent (retailer) observes one customer's state, selects a product to promote (or nothing), and collects revenue. The episode ends when the customer churns. The discount depth on every promoted item is fixed at $\delta$ (e.g. 0.30).

### State Space $\mathcal{S}$

In full generality, the state at time $t$ would comprise user-level features $U_t \in \mathbb{R}^{D_1}$ (demographics, engagement history), product features $P_t \in \mathbb{R}^{N \times D_2}$ (price, category), and user–product interaction features $X_t \in \mathbb{R}^{N \times D_3}$ (purchase counts, affinity scores). For tractability we retain three core components:

$$s_t = (c_t,\; \mathbf{m}_t,\; \mathbf{l}_t) \quad \in \quad \mathcal{S} \cup \{s_{\varnothing}\}$$

| Component | Domain | Meaning |
|-----------|--------|---------|
| $c_t$ | $[0,1]$ | **Churn propensity**: a scalar summarizing overall disengagement risk. Evolves based on customer's recent purchase history and engagement level. |
| $m_t^{(i)}$ | $[0, \infty)$ | **Discount memory** for product $i$: an exponentially weighted accumulation of past discounts. Higher values mean the customer's reference price has been distorted downward. |
| $l_t^{(i)}$ | $\mathbb{Z}_{\ge 0}$ | **Recency**: number of periods since the customer last purchased product $i$. |

Here $\mathbf{m}_t = (m_t^{(1)}, \ldots, m_t^{(N)})$ and $\mathbf{l}_t = (l_t^{(1)}, \ldots, l_t^{(N)})$, giving a **live-state** dimension of $1 + 2N$.

**Terminal state** $s_{\varnothing}$: An absorbing state representing a churned customer. Once $s_t = s_{\varnothing}$, the customer is permanently lost: no further purchases occur and $R(s_{\varnothing}, \cdot) = 0$ for all actions. The episode ends.

### Action Space $\mathcal{A}$

$$a_t \in \mathcal{A} = \{0, 1, 2, \ldots, N\}$$

$a_t = 0$ means no promotion. $a_t = i$ means promote product $i$ at a discounted price $(1 - \delta) \cdot p_i$.

### Transition Dynamics $P$

The state evolves via the following mechanisms:

**1. Effective price.** The price the customer faces for product $j$:

$$\tilde{p}_j(a_t) = p_j \cdot \bigl(1 - \delta \cdot \mathbb{1}[a_t = j]\bigr)$$

**2. Reference price and perceived deal.** Past discounts erode what the customer considers a "fair" price. The reference price for product $j$ is:

$$r_t^{(j)} = p_j - \beta_m \cdot m_t^{(j)}$$

The customer's purchase decision is driven by the *perceived deal* — the gap between their reference price and the actual price:

$$d_t^{(j)} = r_t^{(j)} - \tilde{p}_j(a_t)$$

When $d > 0$ the customer feels they are getting a bargain. When $d < 0$ the price exceeds expectations and the customer is reluctant to buy.

**3. Purchase probability.** Each product is purchased independently via a logistic model:

$$P(\text{buy}_j \mid s_t, a_t) = \sigma\!\bigl(\,\beta_0^{(j)} + \beta_p \cdot d_t^{(j)} - \beta_l \cdot l_t^{(j)}\bigr)$$

| Parameter | Role |
|-----------|------|
| $\beta_0^{(j)}$ | Baseline popularity of product $j$ |
| $\beta_p > 0$ | Price sensitivity (how strongly the perceived deal drives purchase) |
| $\beta_m > 0$ | Memory sensitivity (how much past discounts distort the reference price) |
| $\beta_l > 0$ | Recency decay (products not bought recently are harder to sell) |

**4. Memory update.**

$$m_{t+1}^{(i)} = \alpha \, m_t^{(i)} + \mathbb{1}[a_t = i] \cdot \delta, \qquad \alpha \in [0,1)$$

Memory decays at rate $\alpha$ each period and receives a $\delta$ boost whenever the product is promoted.

**5. Recency update.**

$$l_{t+1}^{(i)} = \begin{cases} 0 & \text{if product } i \text{ was purchased at } t \\ l_t^{(i)} + 1 & \text{otherwise} \end{cases}$$

**6. Churn propensity update.** The scalar $c_t$ captures global disengagement and evolves based on whether the customer made *any* purchase this period:

$$c_{t+1} = \begin{cases} \max(c_t - \eta,\; 0) & \text{if any product was purchased at } t \\ \min(c_t + \eta,\; 1) & \text{otherwise} \end{cases}$$

where $\eta > 0$ controls how quickly engagement builds or erodes. Purchases reduce churn risk; idle periods increase it.

**7. Churn (transition to terminal state).** At the end of each step, the customer churns with probability:

$$P(s_{t+1} = s_{\varnothing} \mid s_t, a_t) = c_t \cdot \prod_{j=1}^{N}\bigl(1 - P(\text{buy}_j \mid s_t, a_t)\bigr)$$

Churn probability is the product of the customer's current disengagement level $c_t$ and the probability that they bought *nothing* this period. If the customer churns, the state transitions to the absorbing terminal state $s_{\varnothing}$ and the episode ends with all future rewards equal to zero.

### Reward Function $R$

$$E[R(s_t, a_t)] = \begin{cases} \displaystyle\sum_{j=1}^{N} P(\text{buy}_j \mid s_t, a_t) \cdot \tilde{p}_j(a_t) & \text{if } s_t \neq s_{\varnothing} \\[6pt] 0 & \text{if } s_t = s_{\varnothing} \end{cases}$$

For live states, the immediate reward is expected revenue. Discounting has two distinct immediate costs:
- **Cannibalization**: If the customer would have bought the item anyway at full price, the discount is pure waste — we sacrifice $\delta \cdot p_i$ in margin for a sale that would have occurred regardless.
- **Revenue dilution**: Even when the discount *does* cause an incremental purchase (one that wouldn't have happened at full price), we only earn $(1-\delta) \cdot p_i$ per unit instead of $p_i$.

The long-term cost — **discount addiction** — does not appear in $R$ directly. It flows through the state transition: promoting product $i$ raises $m_{t+1}^{(i)}$, which lowers the future reference price $r_{t+1}^{(i)}$, which reduces future full-price purchase probability. Similarly, **churn** is penalized indirectly: allowing $c_t$ to rise increases the probability of transitioning to $s_{\varnothing}$, which zeroes out all future revenue.

### Discount Factor and Objective

We use $\gamma \in (0,1)$ (e.g. $\gamma = 0.99$). The objective is:

$$\max_{\pi}\; \mathbb{E}_{\pi}\!\left[\sum_{t=0}^{\infty} \gamma^t \, R(s_t, a_t)\right]$$

Combined with stochastic churn, $\gamma$ gives an effective planning horizon that depends on customer engagement. A low-churn customer has a longer horizon, making the long-term addiction cost more salient to the optimal policy.

### Example

To build intuition, consider a store with $N = 2$ products and these parameters:

**Model parameters:**

| | Coffee ($j=1$) | Tea ($j=2$) |
|--|:--:|:--:|
| Shelf price $p_j$ | \$10 | \$5 |
| Baseline $\beta_0^{(j)}$ | 0.5 | 0.3 |

$\delta = 0.30$, $\alpha = 0.90$, $\beta_m = 2$, $\beta_p = 0.5$, $\beta_l = 0.05$, $\eta = 0.05$, $\gamma = 0.99$

**Initial state:**

| | Coffee ($j=1$) | Tea ($j=2$) |
|--|:--:|:--:|
| Memory $m_0^{(j)}$ | 1.5 (many past discounts) | 0.0 (never discounted) |
| Recency $l_0^{(j)}$ | 2 periods | 8 periods |

$c_0 = 0.05$

---

**Suppose the agent promotes Coffee** ($a_0 = 1$):

**Step 1 — Effective prices:**
- Coffee: $\tilde{p}_1 = 10 \times (1 - 0.30) = \$7.00$
- Tea: $\tilde{p}_2 = \$5.00$ (no discount)

**Step 2 — Reference prices:**
- Coffee: $r_1 = 10 - 2 \times 1.5 = \$7.00$ (eroded by past discounts)
- Tea: $r_2 = 5 - 2 \times 0 = \$5.00$ (undistorted)

**Step 3 — Perceived deals:**
- Coffee: $d_1 = 7 - 7 = 0$ (discount merely restores "neutral" — no real bargain)
- Tea: $d_2 = 5 - 5 = 0$ (also neutral at full price)

**Step 4 — Purchase probabilities:**
- Coffee: $\sigma(0.5 + 0.5 \times 0 - 0.05 \times 2) = \sigma(0.4) \approx 0.60$
- Tea: $\sigma(0.3 + 0.5 \times 0 - 0.05 \times 8) = \sigma(-0.1) \approx 0.48$

**Step 5 - Env update:**
- $P(\text{churn}) = 0.05 \times (1 - 0.60) \times (1 - 0.48) \approx 1\%$
- Suppose the customer buys coffee and does not churn
- $c_{1} = \max(c_0 - \eta, 0) = \max(0.05 - 0.05, 0) = 0$

**Step 6 — Reward:**
- Expected revenue: $E(R_0) = \$7.00 \times 0.60 + \$5.00 \times 0.48 \approx \$6.60$

**Step 7 — State transition:**
- Coffee memory: $m_{1}^{(1)} = 0.9 \times 1.5 + 0.30 = 1.65$ *(addiction deepens)*
- Tea memory: $m_{1}^{(2)} = 0.9 \times 0 = 0.00$
- Coffee recency: $l_1^{(1)}= 0$ *(product was purchased)*
- Tea recency: $l_1^{(2)}= l_0^{(2)} + 1 = 9$ *(product was not purchased)*

$s_1 = (c_1,\; \mathbf{m}_1,\; \mathbf{l}_1) = (0.0,\; (1.65,\; 0.0),\; (0,\; 9))$

---

**Now suppose instead the agent promotes Tea** ($a_0 = 2$):

| | Coffee | Tea |
|--|:--:|:--:|
| Effective price | $10.00 | 5 \times (1 - 0.30) = \$3.50 |
| Reference price | 10 - 2 \times 1.5 = \$7.00 | 5 - 2 \times 0 = \$5.00 |
| Perceived deal | 7.00 - 10 = -3.00 | 5 - 3.50 = +1.50 |
| $P(\text{buy})$ | $\sigma(0.5 - 1.5 - 0.1) = \sigma(-1.1) \approx 0.25$ | $\sigma(0.3 + 0.75 - 0.4) = \sigma(0.65) \approx 0.66$ |

$E(R_1) = \$3.50 \times 0.66 + 10 \times 0.25 \approx \$4.81$

**Key insight:** Promoting Coffee earned more immediate expected revenue (\$6.60 vs \$4.81), but it deepened Coffee addiction ($m_1^{(1)} \to 1.65$). Promoting Tea sacrifices short-term revenue, but started building a new relationship and earned a genuine "deal" lift. Meanwhile, Coffee at full price had $d = -3$ — drastically reducing purchase probability at full price.

This is the fundamental trade-off the optimal policy must navigate: short-term revenue vs. long-term reference-price erosion.

## Problem Versions

We define three versions of increasing complexity per the CME 241 project guidelines.

### Version A: Ideal / Commercial

The full problem with no simplifications:
- **Products:** Full catalog ($N \sim 100{+}$ SKUs), continuous prices.
- **State:** Rich user and product features (demographics, engagement history, purchase history, cross-product affinities), real-time seasonality, cost models.
- **Transitions:** Learned from production transaction logs; non-stationary (customer preferences drift).
- **Action:** Relax the fixed-$\delta$ constraint to include regional or time-varying discount depths.
- **Scale:** Millions of customers, personalized policies per user segment.

This version requires a production ML pipeline and is beyond the scope of the course.

### Version B: Phase 3 Target (RL)

A realistic version solvable with function approximation RL:
- **Products:** $N \approx 5$ product categories (clustered from Dunnhumby data).
- **State:** Continuous $(c_t, \mathbf{m}_t, \mathbf{l}_t)$.
- **Action:** $\mathcal{A} = \{0, 1, \ldots, 5\}$ (promote one category or nothing).
- **Transitions:** Simulated customer; the model-free agent does **not** have access to $P$.

| Parameter | Source | Details |
|--|:--:|--|
| $\delta,\; \gamma$ | Fixed | Design choices ($\delta = 0.30$, $\gamma = 0.99$) |
| $p_j,\; \beta_0^{(j)}$ | Calibrated | Per-category prices and baselines fitted from Dunnhumby data |
| $\beta_p,\; \beta_m,\; \beta_l$ | Calibrated | Demand-model coefficients estimated from purchase history data |
| $\alpha,\; \eta$ | Calibrated | Behavioral dynamics estimated from data |
| $\pi^*$ | **Learned** | Policy discovered via RL |

### Version C: Phase 2 Target (DP)

A simplified version solvable via tabular Dynamic Programming:
- **Products:** $N \leq 5$ product categories (we use $N = 2$ for the worked example). Capping $N$ keeps the $2^N$ purchase-subset enumeration tractable ($2^5 = 32$ outcomes per action).
- **State:** Discretized - $c_t \in \{\text{Low}, \text{High}\}$, $m_t^{(i)} \in \{\text{Low}, \text{Med}, \text{High}\}$, $l_t^{(i)} \in \{\text{Recent}, \text{Stale}\}$ where "Stale" captures $l_t$ being greater than some threshold, giving $|\mathcal{S}| = 2 \times 3^N \times 2^N$ states (72 for $N=2$).
- **Action:** $\mathcal{A} = \{0, 1, \ldots, N\}$.
- **Purchase model:** Same independent logistic model as the general formulation. Each product $j$ is bought independently with probability $\sigma(u_j)$, so every subset $B \subseteq \{1,\ldots,N\}$ is a possible outcome. This yields at most $2^N + 1$ transition branches per $(s, a)$ pair (one per purchase subset, plus churn when $B = \varnothing$).
- **Transitions:** Tabular $P(s' \mid s, a)$ assumed known and static. Computed **analytically** from the demand model.

| Parameter | Source | Details |
|--|:--:|--|
| All model params | Fixed | Hand-chosen values (as in the worked example above) |
| $P(s' \mid s, a)$ | **Computed** | Full transition matrix derived analytically from the demand model |
| $\pi^*$ | **Computed** | Exact solution via Value Iteration |

This version is small enough for exact computation while still exhibiting the core churn-vs-addiction trade-off.

## Code

In [ ]:
from __future__ import annotations

import numpy as np
from dataclasses import dataclass
from itertools import product as cartesian_product
from typing import FrozenSet, Iterable, Tuple

from rl.distribution import Categorical, Distribution
from rl.markov_process import NonTerminal, Terminal, State
from rl.markov_decision_process import MarkovDecisionProcess

# ---------------------------------------------------------------------------
# Phase 2 (Version C): Tabular DP MDP for Discount Targeting
# ---------------------------------------------------------------------------

@dataclass(frozen=True)
class DiscountState:
    """Discretized customer state for the DP version.

    Attributes:
        churn_propensity: Index into C_GRID  (0 = Low, 1 = High)
        discount_memory:  Tuple of per-product indices into M_GRID
                          (0 = Low, 1 = Med, 2 = High)
        purchase_recency: Tuple of per-product indices into L_GRID
                          (0 = Recent, 1 = Stale)
    """
    churn_propensity: int
    discount_memory: Tuple[int, ...]
    purchase_recency: Tuple[int, ...]


@dataclass(frozen=True)
class Product:
    id: int
    price: float
    baseline_demand: float


class DiscountDP(MarkovDecisionProcess[DiscountState, int]):
    """Version C MDP: independent-purchase model, discretized state.

    Each product is purchased independently (logistic probability), so
    every subset of products is a possible outcome.  We cap N <= 5 to
    keep the 2^N subset enumeration tractable (at most 32 outcomes per
    action).  Churn occurs only when no product is purchased.
    """

    MAX_N = 5  # keeps 2^N enumeration tractable

    def __init__(self, products: list[Product]):
        self.N: int = len(products)
        assert self.N <= self.MAX_N, \
            f"N={self.N} exceeds MAX_N={self.MAX_N}; " \
            f"2^N={2**self.N} purchase subsets would be too large for tabular DP"

        # Model parameters (hand-chosen for Phase 2)
        self.delta: float = 0.30        # Fixed discount depth
        self.gamma: float = 0.99        # MDP discount factor
        self.beta_p: float = 0.5        # Price sensitivity
        self.beta_m: float = 2.0        # Memory sensitivity
        self.beta_l: float = 0.05       # Recency decay
        self.alpha: float = 0.9         # Memory decay rate
        self.eta: float = 0.05          # Churn volatility

        # Per-product parameters (0-indexed)
        self.prices = [p.price for p in products]
        self.baselines = [p.baseline_demand for p in products]

        # Discretization grids
        self.C_GRID = [0.05, 0.50]       # Low / High churn
        self.M_GRID = [0.0, 1.0, 2.0]    # Low / Med / High memory
        self.L_GRID = [0, 5]             # Recent / Stale

    # ------------------------------------------------------------------
    # MarkovDecisionProcess interface
    # ------------------------------------------------------------------

    def actions(self, state: NonTerminal[DiscountState]) -> Iterable[int]:
        """0 = no promotion, 1..N = promote product j."""
        return list(range(self.N + 1))

    def step(
        self,
        state: NonTerminal[DiscountState],
        action: int,
    ) -> Distribution[Tuple[State[DiscountState], float]]:
        s: DiscountState = state.state          # unwrap inner state

        # 1. Decode discretized indices → continuous values
        c_val = self.C_GRID[s.churn_propensity]
        m_vals = [self.M_GRID[s.discount_memory[j]] for j in range(self.N)]
        l_vals = [self.L_GRID[s.purchase_recency[j]] for j in range(self.N)]

        # 2. Compute per-product buy probability and effective price
        buy_probs: list[float] = []
        eff_prices: list[float] = []

        for j in range(self.N):
            is_promoted = (action == j + 1)     # actions are 1-indexed
            eff_p = self.prices[j] * (1.0 - self.delta) if is_promoted \
                else self.prices[j]
            eff_prices.append(eff_p)

            ref_price = self.prices[j] - self.beta_m * m_vals[j]
            deal = ref_price - eff_p

            logit = self.baselines[j] + self.beta_p * deal \
                - self.beta_l * l_vals[j]
            buy_probs.append(1.0 / (1.0 + np.exp(-logit)))

        # 3. Enumerate all 2^N purchase subsets
        outcomes: dict[Tuple[State[DiscountState], float], float] = {}

        for bought_vec in cartesian_product([False, True], repeat=self.N):
            # Joint probability of this particular buy/no-buy combination
            prob = 1.0
            for j in range(self.N):
                prob *= buy_probs[j] if bought_vec[j] else (1.0 - buy_probs[j])
            if prob <= 0:
                continue

            purchased: FrozenSet[int] = frozenset(
                j for j in range(self.N) if bought_vec[j]
            )
            reward = sum(eff_prices[j] for j in purchased)

            if len(purchased) > 0:
                # At least one product bought → no churn
                next_s = self._next_state(s, action, purchased)
                key = (NonTerminal(next_s), reward)
                outcomes[key] = outcomes.get(key, 0.0) + prob
            else:
                # Nothing bought → churn with probability c_val
                p_churn = prob * c_val
                if p_churn > 0:
                    outcomes[(Terminal(s), 0.0)] = \
                        outcomes.get((Terminal(s), 0.0), 0.0) + p_churn

                # Nothing bought → survive
                p_survive = prob * (1.0 - c_val)
                if p_survive > 0:
                    next_s = self._next_state(s, action, purchased)
                    key = (NonTerminal(next_s), 0.0)
                    outcomes[key] = outcomes.get(key, 0.0) + p_survive

        return Categorical(outcomes)

    # ------------------------------------------------------------------
    # Helpers
    # ------------------------------------------------------------------

    def _next_state(
        self,
        s: DiscountState,
        action: int,
        purchased: FrozenSet[int],
    ) -> DiscountState:
        """Deterministic next state given the current state, action, and
        which products were purchased.

        Args:
            s:         Current (inner) DiscountState.
            action:    Chosen action (0 = no promo, 1..N = promote product j).
            purchased: Frozenset of 0-based indices of purchased products
                       (empty if nothing was bought).
        """
        # A. Churn propensity: decrease on any purchase, increase on idle
        curr_c = self.C_GRID[s.churn_propensity]
        if len(purchased) > 0:
            new_c_val = max(curr_c - self.eta, 0.0)
        else:
            new_c_val = min(curr_c + self.eta, 1.0)
        new_c_idx = int(np.argmin([abs(new_c_val - g) for g in self.C_GRID]))

        # B. Per-product memory and recency
        new_m: list[int] = []
        new_l: list[int] = []
        for j in range(self.N):
            # Memory: exponential decay + bump if promoted
            curr_m = self.M_GRID[s.discount_memory[j]]
            is_promoted = (action == j + 1)
            next_m_val = self.alpha * curr_m + (self.delta if is_promoted else 0.0)
            new_m.append(int(np.argmin(
                [abs(next_m_val - g) for g in self.M_GRID]
            )))

            # Recency: reset to 0 if bought, else increment
            curr_l = self.L_GRID[s.purchase_recency[j]]
            next_l_val = 0 if j in purchased else curr_l + 1
            new_l.append(int(np.argmin(
                [abs(next_l_val - g) for g in self.L_GRID]
            )))

        return DiscountState(
            churn_propensity=new_c_idx,
            discount_memory=tuple(new_m),
            purchase_recency=tuple(new_l),
        )

## References

1. Dunnhumby. (2017). *The Complete Journey: Household Analytics and Marketing Impact.*